In [2]:
import tensorflow as tf
import os
import numpy as np
import medmnist

/Users/sanjeevnara/miniforge3/envs/tf_Dev/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading the Video Data
**Organ MNIST 3D (Medical Video Data for Classification)** <br/>
Video Data Shape: 28 frames x (28 x 28 x 1) per frame <br/>
Number of Classes: 11

In [3]:
# DATA PARAMETERS
DATASET_NAME = "organmnist3d"
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (28, 28, 28, 1)
NUM_CLASSES = 11

# Utility Function to get data
def download_and_prepare_dataset(data_info):
    """Utility function to download the dataset.

    Arguments:
        data_info (dict): Dataset metadata.
    """
    data_path = tf.keras.utils.get_file(origin=data_info["url"], md5_hash=data_info["MD5"])

    # Use numpy to load data 
    with np.load(data_path) as data:
        # Get videos
        train_videos = data["train_images"]
        valid_videos = data["val_images"]
        test_videos = data["test_images"]

        # Get labels
        train_labels = data["train_labels"].flatten()
        valid_labels = data["val_labels"].flatten()
        test_labels = data["test_labels"].flatten()

    return (
        (train_videos, train_labels),
        (valid_videos, valid_labels),
        (test_videos, test_labels),
    )

# Get the metadata of the dataset
info = medmnist.INFO[DATASET_NAME]

# Get the dataset
prepared_dataset = download_and_prepare_dataset(info)
(train_videos, train_labels) = prepared_dataset[0]
(valid_videos, valid_labels) = prepared_dataset[1]
(test_videos, test_labels) = prepared_dataset[2]

In [4]:
print(train_videos[0].shape, type(train_videos[0]))
print(train_labels[0].shape, type(train_labels[0]))

(28, 28, 28) <class 'numpy.ndarray'>
() <class 'numpy.uint8'>


### Create a tf.Data pipeline for the model

Prepare the Dataloaders for training. We use a batch size of 32:

In [5]:
@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """Preprocess the frames tensors and parse the labels."""
    # Preprocess images
    frames = tf.image.convert_image_dtype(
        frames[
            ..., tf.newaxis
        ],  # The new axis is to help for further processing with Conv3D layers
        tf.float32,
    )
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label

def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader

trainloader = prepare_dataloader(train_videos, train_labels, "train")
validloader = prepare_dataloader(valid_videos, valid_labels, "valid")
testloader = prepare_dataloader(test_videos, test_labels, "test")

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-01-28 15:59:33.373729: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-28 15:59:33.373875: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
for frames,label in trainloader.take(1):
    print(frames.shape)
    print(label.shape)

(32, 28, 28, 28, 1)
(32,)


2023-01-28 15:59:33.481379: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


## Define and Compile the model

We first implement all the necessary layer blocks of the model:

In [10]:
# Tublet Embedding Layer to extract volume patches from the video and convert to embeddings
class TubeletEmbedding(tf.keras.layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = tf.keras.layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = tf.keras.layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches

# Positional Encoder to add positional information to embeddings
class PositionalEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = tf.keras.layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens

In [13]:
# HYPERPARAMETERS
# OPTIMIZER
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 60

# TUBELET EMBEDDING
PATCH_SIZE = (8, 8, 8) # 3D patches (volumes)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2 # We assume square input

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128 # Embedding Dim
NUM_HEADS = 8
NUM_LAYERS = 8

# Helper function to initialize the model
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    # Get the input layer
    inputs = tf.keras.layers.Input(shape=input_shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # Encode patches.
    encoded_patches = positional_encoder(patches)

    # Create multiple layers of the Transformer block:
    # input
    # |      \ 
    # |       \
    # MSA     |
    # |       /
    # Skip(+)
    # |       \
    # |        \
    # LayerNorm \
    # |          |
    # GeLU MLP(embed_dim x 4, embed_dim)
    # |          |
    # |        /
    # |       /
    # Skip(+) 
    # |
    # Output  

    for _ in range(transformer_layers):
        # Layer normalization and MHSA
        x1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
        )(x1, x1)

        # Skip connection
        x2 = tf.keras.layers.Add()([attention_output, encoded_patches])

        # Layer Normalization and MLP
        x3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = tf.keras.Sequential(
            [
                tf.keras.layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                tf.keras.layers.Dense(units=embed_dim, activation=tf.nn.gelu),
            ]
        )(x3)

        # Skip connection
        encoded_patches = tf.keras.layers.Add()([x3, x2])

    # Layer normalization and Global average pooling.
    representation = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)(encoded_patches)
    representation = tf.keras.layers.GlobalAvgPool1D()(representation)

    # Classify outputs.
    outputs = tf.keras.layers.Dense(units=num_classes, activation="softmax")(representation)

    # Create the Keras model.
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

## Train the model

In [14]:
# Initialize model
model = create_vivit_classifier(
    tubelet_embedder=TubeletEmbedding(
        embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
    ),
    positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
)
# Compile the model with the optimizer, loss function
# and the metrics.
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=[
        tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
        tf.keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
    ],
)
# Train the model.
_ = model.fit(trainloader, epochs=EPOCHS, validation_data=validloader)

_, accuracy, top_5_accuracy = model.evaluate(testloader)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")
print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

Epoch 1/60


2023-01-28 16:00:41.184253: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - ETA: 0s - loss: 2.4913 - accuracy: 0.1070 - top-5-accuracy: 0.5576

2023-01-28 16:00:53.625736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


31/31 [==============================] - 19s 381ms/step - loss: 2.4913 - accuracy: 0.1070 - top-5-accuracy: 0.5576 - val_loss: 2.4382 - val_accuracy: 0.0870 - val_top-5-accuracy: 0.5776
Epoch 2/60
31/31 [==============================] - 6s 202ms/step - loss: 2.2656 - accuracy: 0.1543 - top-5-accuracy: 0.6626 - val_loss: 2.2604 - val_accuracy: 0.2484 - val_top-5-accuracy: 0.6832
Epoch 3/60
31/31 [==============================] - 6s 201ms/step - loss: 2.0993 - accuracy: 0.2263 - top-5-accuracy: 0.7603 - val_loss: 1.8563 - val_accuracy: 0.2857 - val_top-5-accuracy: 0.8882
Epoch 4/60
31/31 [==============================] - 6s 207ms/step - loss: 1.8830 - accuracy: 0.2757 - top-5-accuracy: 0.8519 - val_loss: 1.5766 - val_accuracy: 0.3106 - val_top-5-accuracy: 0.8944
Epoch 5/60
31/31 [==============================] - 6s 205ms/step - loss: 1.6511 - accuracy: 0.3693 - top-5-accuracy: 0.8971 - val_loss: 1.3644 - val_accuracy: 0.4161 - val_top-5-accuracy: 0.9441
Epoch 6/60
31/31 [============

In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 28, 28, 28,  0           []                               
                                 1)]                                                              
                                                                                                  
 tubelet_embedding_2 (TubeletEm  (None, 27, 128)     65664       ['input_3[0][0]']                
 bedding)                                                                                         
                                                                                                  
 positional_encoder_2 (Position  (None, 27, 128)     3456        ['tubelet_embedding_2[0][0]']    
 alEncoder)                                                                                   

In [17]:
import ipywidgets
import os
import io
import imageio

## Visualize Results

In [19]:
# Prediction on Test Set and Visualization:
NUM_SAMPLES_VIZ = 25
testsamples, labels = next(iter(testloader))
testsamples, labels = testsamples[:NUM_SAMPLES_VIZ], labels[:NUM_SAMPLES_VIZ]

ground_truths = []
preds = []
videos = []

for i, (testsample, label) in enumerate(zip(testsamples, labels)):
    # Generate gif
    with io.BytesIO() as gif:
        imageio.mimsave(gif, (testsample.numpy() * 255).astype("uint8"), "GIF", fps=5)
        videos.append(gif.getvalue())

    # Get model prediction
    output = model.predict(tf.expand_dims(testsample, axis=0))[0]
    pred = np.argmax(output, axis=0)

    ground_truths.append(label.numpy().astype("int"))
    preds.append(pred)


def make_box_for_grid(image_widget, fit):
    """Make a VBox to hold caption/image for demonstrating option_fit values.
    """
    # Make the caption
    if fit is not None:
        fit_str = "'{}'".format(fit)
    else:
        fit_str = str(fit)

    h = ipywidgets.HTML(value="" + str(fit_str) + "")

    # Make the green box with the image widget inside it
    boxb = ipywidgets.widgets.Box()
    boxb.children = [image_widget]

    # Compose into a vertical box
    vb = ipywidgets.widgets.VBox()
    vb.layout.align_items = "center"
    vb.children = [h, boxb]
    return vb


boxes = []
for i in range(NUM_SAMPLES_VIZ):
    ib = ipywidgets.widgets.Image(value=videos[i], width=100, height=100)
    true_class = info["label"][str(ground_truths[i])]
    pred_class = info["label"][str(preds[i])]
    caption = f"T: {true_class} | P: {pred_class}"

    boxes.append(make_box_for_grid(ib, caption))

ipywidgets.widgets.GridBox(
    boxes, layout=ipywidgets.widgets.Layout(grid_template_columns="repeat(5, 200px)")
)

1/1 [==============================] - 0s 66ms/step


GridBox(children=(VBox(children=(HTML(value="'T: pancreas | P: bladder'"), Box(children=(Image(value=b'GIF89a\…